# Prérequis

Premièrement, nous avons besoin d'importer les bibliothèques python nécessaires pour notre algorithme.

La bibliothèque la plus importante est py_entitymatching, qui est développée par Anhais Group dans le cadre du projet Magellan.

https://sites.google.com/site/anhaidgroup/home

In [ ]:
# importation des bibliothèques requises (notamment py_entitymatching)

!pip install unidecode
!pip install fuzzywuzzy
!pip install missingno
!pip uninstall Cython -y
!pip uninstall py_entitymatching -y
!pip uninstall py_stringsimjoin -y
!pip install py_stringsimjoin==0.3.3
!pip install py_entitymatching==0.4.1


import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
from fuzzywuzzy import fuzz
from unidecode import unidecode
import missingno as msno
import py_stringsimjoin as ssj
import py_stringmatching as sm
import py_entitymatching as em
import math

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.1 MB/s eta 0:00:00
Found existing installation: Cython 3.0.8
Uninstalling Cython-3.0.8:
  Successfully uninstalled Cython-3.0.8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for py_stringsimjoin: filename=py_stringsimjoin-0.3.3-cp310-cp310-linux_x86_64.whl size=4517644 sha256=10b304cf54996ac533c738ea9d63010758962d3afb67414bceb7fc014097635f
  Stored in directory: /root/.cache/pip/wheels/63/47/e6/f221c81b7b7da486f12483e0e1e766e8be3b1c25e1f08a9203
  Created wheel for py_stringmatching: filename=py_stringmatching-0.4.5-cp310-cp310-linux_x86_64.whl size=2833559 sha256=4d26ac6286cec6a23880e8d50e72bbbf368637e66af345807aabdb837a2763c6
  Stored in directory: /root/.cache/pip/wheels/c2/8b/ee/728d0fd5aaf2230776060644dd45e2ef

--------

Le but des cellules ci dessous est de créer une fonction parsing_address afin de séparer le nom de la voie ("Avenue") du type de la voie ("du Général De Gaulle").

Nous devons avoir une nomenclature homogène entre l'adresse de BAN et de IPE, c'est pourquoi nous séparons le type de voie du nom de voie.

Une autre possibilité de cette fonction est aussi de repérer les abréviations de type de voie et de les convertir en nom complet (Av --> Avenue). Ces abréviations sont listées dans abbreviations_list, et le nom complet de voie dans fullname_list


In [ ]:
abbreviations_list = [
    'ABE', 'ACH', 'AER', 'AIRE', 'ALL', 'ANSE', 'AR', 'ARC', 'ART', 'AUT', 'AV', 'BAL', 'BAS', 'BASE', 'BAT', 'BCH', 'BCLE', 'BD', 'BER', 'BOIS', 'BRC', 'BRE', 'BRG', 'BRTL', 'BSTD',
    'BUT', 'CALE', 'CAMP', 'CAN', 'CAR', 'CARE', 'CARR', 'CASR', 'CAU', 'CC', 'CCAL', 'CD', 'CEIN', 'CGNE', 'CHE', 'CHEM', 'CHEZ', 'CHI', 'CHL', 'CHP', 'CHS', 'CHT', 'CITE', 'CLOS', 'COL',
    'COLI', 'COR', 'CORO', 'COT', 'COTE', 'COTT', 'COUR', 'CPG', 'CR', 'CRS', 'CST', 'CTR', 'CTRE', 'CVE', 'CHV', 'D', 'DIG', 'DOM', 'DSC', 'ECA', 'ECL', 'EGL', 'EN', 'ENC', 'ENV', 'ESC',
    'ESP', 'ESPA', 'FG', 'FON', 'FORM', 'FORT', 'FOS', 'FRM', 'GAL', 'GARE', 'GBD', 'GPE', 'GPL', 'GPT', 'GR', 'GRI', 'HAM', 'HCH', 'HIP', 'HLE', 'HLG', 'HLM', 'ILE', 'ILOT', 'IMM', 'IMP',
    'JARD', 'JTE', 'LD', 'LEVE', 'LOT', 'MAIL', 'MAIS', 'MAR', 'MAS', 'MLN', 'MTE', 'MUS', 'NTE', 'PAE', 'PAL', 'PARC', 'PAS', 'PASS', 'PAT', 'PAV', 'PCH', 'PERI', 'PIM', 'PISTE', 'PKG', 'PL',
    'PLAN', 'PLCI', 'PLE', 'PLN', 'PLT', 'PN', 'PNT', 'PONT', 'PORT', 'POT', 'PR', 'PRE', 'PROM', 'PRQ', 'PRT', 'PRV', 'PTA', 'PTE', 'PTTE', 'QU', 'QUA', 'R', 'RAC', 'RAID', 'RDE', 'REM', 'RES',
    'RLE', 'RN', 'RNG', 'ROC', 'RPE', 'RPT', 'RTD', 'RTE', 'RUELLETTE', 'RUETTE', 'SEN', 'SENTE', 'SQ', 'STA', 'STDE', 'TOUR', 'TPL', 'TRA', 'TRN', 'TRT', 'TSSE', 'VAL', 'VC', 'VCHE', 'VEN', 'VIA',
    'VLA', 'VLGE', 'VOI', 'VR', 'VTE', 'ZA', 'ZAC', 'ZAD', 'ZI', 'ZONE', 'ZUP'
]
#abbreviations = pd.DataFrame({'Abbreviation': abbreviations_list})

In [ ]:
fullname_list = [
    'Abbaye', 'Ancien chemin', 'Aéroport', 'Aire', 'Allée', 'Anse',
    'Ancienne Rue', 'Arcade', 'Ancienne route', 'Autoroute', 'Avenue', 'Balcon',
    'Bastion', 'Base', 'Bâtiment', 'Bas Chemin', 'Boucle', 'Boulevard', 'Berge',
    'Bois', 'Brèche', 'Barrière', 'Bourg', 'Bretelle', 'Bastide', 'Butte', 'Cale',
    'Camp', 'Canal', 'Carrefour', 'Carrière', 'Carrè', 'Caserne', 'Carreau',
    'Chemin communal', 'Centre Commercial', 'Chemin départemental', 'Ceinture',
    'Campagne', 'Chemin', 'Cheminement', 'Chez', 'Charmille', 'Chalet', 'Chapelle',
    'Chaussée', 'Château', 'Cité', 'Clos', 'Col', 'Colline', 'Corniche', 'Coron', 'Côte',
    'Coteau', 'Cottage', 'Cour', 'Camping', 'Chemin rural', 'Cours', 'Castel',
    'Contour', 'Centre', 'Coursive', 'Chemin Vicinal', 'Départementale', 'Digue',
    'Domaine', 'Descente', 'Écart', 'Écluse', 'Église', 'Enceinte', 'Enclos',
    'Enclave', 'Escalier', 'Esplanade', 'Espace', 'Faubourg', 'Fontaine', 'Forum',
    'Fort', 'Fosse', 'Ferme', 'Galerie', 'Gare', 'Grand boulevard', 'Groupe',
    'Grande Place', 'Groupement', 'Grande rue', 'Grille', 'Hameau', 'Haut Chemin',
    'Hippodrome', 'Halle', 'Halage', 'HLM', 'Île', 'Îlot', 'Immeuble', 'Impasse',
    'Jardin', 'Jetée', 'Lieu dit', 'Levée', 'Lotissement', 'Mail', 'Maison', 'Marché',
    'Mas', 'Moulin', 'Montée', 'Musée', 'Nouvelle Route', 'Petite Avenue', 'Palais',
    'Parc', 'Passage', 'Passe', 'Patio', 'Pavillon', 'Petit Chemin', 'Périphérique',
    'Petite Impasse', 'Piste', 'Parking', 'Place', 'Plan', 'Placis', 'Passerelle',
    'Plaine', 'Plateau', 'Passage à Niveau', 'Pointe', 'Pont', 'Port', 'Poterne',
    'Petite Rue', 'Pré', 'Promenade', "Presqu'île", 'Petite Route', 'Parvis',
     'Petite Allée', 'Porte', 'Placette', 'Quai', 'Quartier', 'Rue', 'Raccourci',
     'Raidillon', 'Ronde', 'Rempart', 'Résidence', 'Ruelle', 'Route Nationale',
    'Rangée', 'Rocade', 'Rampe', 'Rond Point', 'Rotonde', 'Route', 'Ruellette',
    'Ruette', 'Sentier', 'Sente', 'Square', 'Station', 'Stade', 'Tour', 'Terre Plein',
    'Traverse', 'Terrain', 'Tertre', 'Terrasse', 'Vallée', 'Voie Communale',
    'Vieux Chemin', 'Venelle', 'Via', 'Villa', 'Village', 'Voie', 'Vieille Rue',
    'Vieille Route', 'Zone Artisanale', 'Zone Aménagement Concerté',
    'Zone Aménagement Différé', 'Zone Industrielle', 'Zone', 'Zone à Urbaniser en Priorité'
]
fullname_list = [string.lower() for string in fullname_list] # Lower Case the fullname of roadtype
fullname_list = [unidecode(string) for string in fullname_list] # Remove accents from words

#abbreviations = pd.DataFrame({'Abbreviation': abbreviations_list})

In [ ]:
abbreviation_to_type_mapping = dict(zip(abbreviations_list, fullname_list))

In [ ]:
def find_string_contains_multi_words(string_list, nb_words):
  #This function to filter the full name of road type that have more than 1 word
  result_list = []
  for string in string_list:
    words = string.split()
    if len(words) == nb_words:
      result_list.append(string)
  return result_list

find_string_contains_2_words =  find_string_contains_multi_words(fullname_list, 2)
find_string_contains_3_words =  find_string_contains_multi_words(fullname_list, 3)


def compare_2_strings(string_list, words, threshold = 75):
  # Calculate the similarity score using Fuzzy Logic of the selected words with each string of the list
  # We just take into account the score that is higher than threshold
  # The highest score corresponds to the word considered to be the best match in the list
  # If no scores exceed the threshold, return None
  best_match = None
  best_score = 0
  for item in string_list:
        similarity_score = fuzz.ratio(words, item)
        if similarity_score >= threshold and similarity_score > best_score:
          best_match = item
          best_score = similarity_score
  return best_match, best_score

def parsing_address(input_string, words_to_compare = 3):
  # This is the recursive function

  words = input_string.split() #Split the address into list of string

  if len(words) > 1:
    nb_words = min(words_to_compare, len(words) - 1)  #Take into account the number of words to compare
  else:
    return '',input_string
  compared_words =' '.join(words[:nb_words])

  if words[0] == "le" or words[0] == "la": # Address start with le/la easily confuse with ile, villa
    return '', input_string                # So we assign these addresses to Nom de voie

  if nb_words == 1:
  #Create the stop condition for recursive function (the base case)
  # In case we need to compare first 1 word of the address, we will compare it with string in abbreviation list and fullname_list
  # After having compared that word with 2 lists, we will see that the word is more similar to which string in which list (to Return it correctly)
    match_abbre, match_score_abbre = compare_2_strings(abbreviations_list, compared_words.upper()) # Uppercase the word when compare with abbreviation list because the string in this list is uppercased
    match_full , match_score_full = compare_2_strings(fullname_list, unidecode(compared_words.lower())) # Lowercase the word and then remove the accents
    if match_abbre != None or match_full != None:
      if match_score_abbre > match_score_full:
        return abbreviation_to_type_mapping[match_abbre], ' '.join(words[1:])
      else:
        return match_full,  ' '.join(words[1:])
    else:
      return '', input_string # If no string in both lists matches that word then Invalid Address or there 's no type of the road in this address

  else:
     # If type of the roads contain 2 or 3 words then
     # we need to compare the first 2,3 words of the input string with the string in fullname_list
     # If Match, stop the function immediately
     # If not, decrease the nb of words for comparison from 3 to 2 or 2 to 1.
    reference_list = find_string_contains_2_words if nb_words == 2 else find_string_contains_3_words
    match_word, match_score = compare_2_strings(reference_list, unidecode(compared_words.lower()))
    if match_word == None :
      return parsing_address(input_string, nb_words - 1)
    else:
      return match_word,  input_string.replace(compared_words,"")

Ainsi, la fonction parsing_address prend en entrée un élément de la colonne "nom_voie" de BAN et le convertit en élément de "TypeVoieImmeuble" et "NomVoieImmeuble" de IPE.

---------

# Preprocessing

Dans cette partie, nous allons devoir obtenir un schéma de données similaire entre la table BAN et la table IPE.
Ainsi, nous devons avoir les mêmes colonnes (notamment séparer le type de voie du nom de voie dans BAN) ainsi qu'avoir des noms de colonnes identiques.

De plus, nous devons nous assurer que le contenu des données est similaires, et donc traiter les majuscules, accents, cartactères spéciaux.

In [ ]:
ipe = pd.read_csv('/content/IPE-AXTD.csv', delimiter=";")
ban = pd.read_csv('/content/BAN-adresses-12.csv', delimiter=";")
# we take ban-adresses-12 as it is the department 12, which is contianed in ipe AXTD region

In [ ]:
# ban['TypeVoie'] = ban['nom_voie'].apply(lambda x : parsing_address(x)[0])
# ban['NomVoieReste'] = ban['nom_voie'].apply(lambda x : parsing_address(x)[1])  #We shouldn't write like this because it will double the processing time

for i in range(len(ban)):
    parsed_address = parsing_address(ban.loc[i, 'nom_voie'])
    #print(i)
    #if parsed_address is not None:
    ban.loc[i, 'TypeVoie'] = parsed_address[0]
    ban.loc[i, 'NomVoieReste'] = parsed_address[1]
ban

,id,id_fantoir,numero,rep,nom_voie,code_postal,code_insee,nom_commune,code_insee_ancienne_commune,nom_ancienne_commune,x,y,lon,lat,type_position,alias,nom_ld,libelle_acheminement,nom_afnor,source_position,source_nom_voie,certification_commune,cad_parcelles,TypeVoie,NomVoieReste
0,12002_0puwwe_00001,NaN,1,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707758.86,6339740.77,3.097003,44.156631,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
1,12002_0puwwe_00003,NaN,3,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707754.93,6339752.98,3.096954,44.156741,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
2,12002_0puwwe_00005,NaN,5,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707758.99,6339764.54,3.097005,44.156845,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
3,12002_0puwwe_00007,NaN,7,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707766.91,6339767.00,3.097104,44.156867,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
4,12002_0puwwe_00009,NaN,9,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707763.40,6339758.66,3.097060,44.156792,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152678,12006_b167_00018,12006_B167,18,NaN,Mas Nespoulous,12430,12006,Alrance,NaN,NaN,675581.14,6334943.59,2.694943,44.113072,entrée,NaN,NaN,ALRANCE,MAS NESPOULOUS,commune,commune,1,120060000E0619,mas,Nespoulous
152679,12006_b167_00020,12006_B167,20,NaN,Mas Nespoulous,12430,12006,Alrance,NaN,NaN,675674.74,6334846.90,2.696117,44.112205,entrée,NaN,NaN,ALRANCE,MAS NESPOULOUS,commune,commune,1,120060000E0743,mas,Nespoulous
152680,12006_b002_00001,12006_B002,1,NaN,La Baraque de Malet,12430,12006,Alrance,NaN,NaN,675308.07,6334466.23,2.691555,44.108766,entrée,NaN,NaN,ALRANCE,LA BARAQUE DE MALET,commune,commune,1,120060000F0538,villa,Baraque de Malet
152681,12006_b002_00003,12006_B002,3,NaN,La Baraque de Malet,12430,12006,Alrance,NaN,NaN,675282.59,6334438.89,2.691238,44.108519,entrée,NaN,NaN,ALRANCE,LA BARAQUE DE MALET,commune,commune,1,120060000F0537,villa,Baraque de Malet


In [ ]:
ban.rename(columns={'typeVoie': 'TypeVoie'}, inplace=True)
ban.rename(columns={'nomVoie': 'NomVoieReste'}, inplace=True)

ban_reduced = ban[['id','numero','rep','TypeVoie','NomVoieReste','code_postal','nom_commune','x','y']]
ipe_reduced = ipe[['IdentifiantImmeuble','NumeroVoieImmeuble','ComplementNumeroVoieImmeuble','TypeVoieImmeuble','NomVoieImmeuble','CodePostalImmeuble','CommuneImmeuble','CoordonneeImmeubleX','CoordonneeImmeubleY']]
ipe_reduced['TypeVoieImmeuble'] = ipe_reduced['TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')


nom_colonnes = {'id': 'IdentifiantImmeuble','numero' : 'NumeroVoieImmeuble','rep' : 'ComplementNumeroVoieImmeuble','TypeVoie' : 'TypeVoieImmeuble','NomVoieReste' : 'NomVoieImmeuble','code_postal' : 'CodePostalImmeuble','nom_commune' : 'CommuneImmeuble','x' : 'CoordonneeImmeubleX','y' : 'CoordonneeImmeubleY'}
ban_reduced = ban_reduced.rename(columns=nom_colonnes)

In [ ]:
ban_reduced

,IdentifiantImmeuble,NumeroVoieImmeuble,ComplementNumeroVoieImmeuble,TypeVoieImmeuble,NomVoieImmeuble,CodePostalImmeuble,CommuneImmeuble,CoordonneeImmeubleX,CoordonneeImmeubleY
0,12002_0puwwe_00001,1,NaN,plan,du Foirail,12520,Aguessac,707758.86,6339740.77
1,12002_0puwwe_00003,3,NaN,plan,du Foirail,12520,Aguessac,707754.93,6339752.98
2,12002_0puwwe_00005,5,NaN,plan,du Foirail,12520,Aguessac,707758.99,6339764.54
3,12002_0puwwe_00007,7,NaN,plan,du Foirail,12520,Aguessac,707766.91,6339767.00
4,12002_0puwwe_00009,9,NaN,plan,du Foirail,12520,Aguessac,707763.40,6339758.66
...,...,...,...,...,...,...,...,...,...
152678,12006_b167_00018,18,NaN,mas,Nespoulous,12430,Alrance,675581.14,6334943.59
152679,12006_b167_00020,20,NaN,mas,Nespoulous,12430,Alrance,675674.74,6334846.90
152680,12006_b002_00001,1,NaN,villa,Baraque de Malet,12430,Alrance,675308.07,6334466.23
152681,12006_b002_00003,3,NaN,villa,Baraque de Malet,12430,Alrance,675282.59,6334438.89


In [ ]:
ipe_reduced

,IdentifiantImmeuble,NumeroVoieImmeuble,ComplementNumeroVoieImmeuble,TypeVoieImmeuble,NomVoieImmeuble,CodePostalImmeuble,CommuneImmeuble,CoordonneeImmeubleX,CoordonneeImmeubleY
0,IMB/12001/X/0001,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06
1,IMB/12001/X/0002,8,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674437.822388,6.361478e+06
2,IMB/12001/X/0003,6,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674408.962423,6.361491e+06
3,IMB/12001/X/0004,10,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674402.615772,6.361449e+06
4,IMB/12001/X/0005,1,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674523.068416,6.361517e+06
...,...,...,...,...,...,...,...,...,...
283826,IMB/48198/X/00TG,5996,NaN,route,du Mas de la Barque,48800,Villefort,773209.199944,6.371295e+06
283827,IMB/48198/X/00TH,6005,NaN,route,du Mas de la Barque,48800,Villefort,773186.659129,6.371244e+06
283828,IMB/48198/X/00TI,5264,NaN,route,du Mas de la Barque,48800,Villefort,773131.114161,6.371255e+06
283829,IMB/48198/X/00TJ,6033,NaN,route,du Mas de la Barque,48800,Villefort,773114.651880,6.371262e+06


Ne pas exécuter cette section à chaque fois, car elle prend beaucoup de temps d'exécution. L'utilisation optimale est de traiter une fois les données brutes de BAN et IPE et ensuite d'enregister les données traitées ban_reduced & ipe_reduced dans des CSV pour les réutiliser plus tard si besoin.

------------

In [ ]:
# ipe = pd.read_csv('/content/IPE-AXTD.csv', delimiter=";")
# ban_reduced = pd.read_csv('/content/ban_decomposed-12.csv')

# ban_reduced = ban_reduced[['id','numero','rep','TypeVoie','NomVoieReste','code_postal','nom_commune','x','y']]
# nom_colonnes = {'id': 'IdentifiantImmeuble','numero' : 'NumeroVoieImmeuble','rep' : 'ComplementNumeroVoieImmeuble','TypeVoie' : 'TypeVoieImmeuble','NomVoieReste' : 'NomVoieImmeuble','code_postal' : 'CodePostalImmeuble','nom_commune' : 'CommuneImmeuble','x' : 'CoordonneeImmeubleX','y' : 'CoordonneeImmeubleY'}
# ban_reduced = ban_reduced.rename(columns=nom_colonnes)

# ipe_reduced = ipe[['IdentifiantImmeuble','NumeroVoieImmeuble','ComplementNumeroVoieImmeuble','TypeVoieImmeuble','NomVoieImmeuble','CodePostalImmeuble','CommuneImmeuble','CoordonneeImmeubleX','CoordonneeImmeubleY']]
# ipe_reduced['TypeVoieImmeuble'] = ipe_reduced['TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')


#si vous avez enregistré les dataframes de la section précédente sou forme de CSV pour les réutiliser, vous pouvez les charger ici en précisant le chemin des CSV.
ban_reduced = pd.read_csv('/content/ban_reduced.csv')
ipe_reduced = pd.read_csv('/content/ipe_reduced.csv')


--------

# Training

Cet algorithme diffère du premier car il est basé sur du machine learning.
Nous lui fournissons des exemples de comparaisons d'adresses (adresses similaires ou non simialires) contenu dans le fichier dataset_annote_V3.csv, et le modèle apprend par lui même les règles de matching. Il s'agit d'un problème de classification binaire en machine learning.
Ainsi, la qualité des prédictions dépendent de la taille, la qualité et l'exaustivité des exemples fournis dans le dataset annoté.

In [ ]:
labeled = pd.read_csv('/content/dataset_annote_V3.csv')
labeled = labeled.dropna(subset=['id','IdentifiantImmeuble'])


labeled = labeled.iloc[:-11,:]

labeled = labeled[~labeled['id'].duplicated()]
labeled = labeled[~labeled['IdentifiantImmeuble'].duplicated()]
labeled = labeled.fillna('')

labeled = labeled.astype({'id': 'string', 'numero' : 'float32' , 'rep' : 'string', 'TypeVoie': 'string', 'NomVoieReste': 'string', 'code_postal': 'int32',
                          'nom_commune': 'string', 'x': 'float32', 'y': 'float32', 'IdentifiantImmeuble': 'string', 'NumeroVoieImmeuble': 'float32',
                          'ComplementNumeroVoieImmeuble': 'string', 'TypeVoieImmeuble': 'string', 'NomVoieImmeuble': 'string',
                          'CodePostalImmeuble': 'int32', 'CommuneImmeuble': 'string', 'CoordonneeImmeubleX': 'float32',
                          'CoordonneImmeubleY': 'float32', 'Label': 'int32'})

A = labeled.iloc[:, :len(labeled.columns)//2]
B = labeled.iloc[:, len(labeled.columns)//2:-1]
labeled

,id,numero,rep,TypeVoie,NomVoieReste,code_postal,nom_commune,x,y,IdentifiantImmeuble,NumeroVoieImmeuble,ComplementNumeroVoieImmeuble,TypeVoieImmeuble,NomVoieImmeuble,CodePostalImmeuble,CommuneImmeuble,CoordonneeImmeubleX,CoordonneImmeubleY,Label
0,12002_0puwwe_00001,1.0,,Plan,du Foirail,12520,Aguessac,707758.8750,6339741.0,IMB/12001/X/0001,4.0,,Impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.1250,6361511.0,0
1,12002_ntujmi_00001,1.0,,Avenue,de la Brezegue,12520,Aguessac,707676.5000,6339929.5,IMB/12001/X/0006,3.0,,Chemin,de Campan,12630,Agen-d'Aveyron,673813.5625,6361964.0,0
2,12002_ntujmi_00002,2.0,,Avenue,de la Brezegue,12520,Aguessac,707691.1250,6339931.5,IMB/12001/X/0007,9.0,,Chemin,de Campan,12630,Agen-d'Aveyron,673709.7500,6361977.5,0
3,12002_ntujmi_00003,3.0,,Avenue,de la Brezegue,12520,Aguessac,707569.0625,6339992.0,IMB/12001/X/000H,14.0,,Chemin,de Cantaserp,12630,Agen-d'Aveyron,673441.2500,6362122.0,0
4,12002_ntujmi_00004,4.0,,Avenue,de la Brezegue,12520,Aguessac,707643.8125,6339963.5,IMB/12001/X/000I,16.0,,Chemin,de Cantaserp,12630,Agen-d'Aveyron,673431.8750,6362076.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,12299_0024_00024,24.0,,Chemin,de la Carral,12430,Villefranche-de-Panat,676497.2500,6331368.0,IMB/12008/X/0001,1.0,,Chemin,de la Carrière,12390,Anglars-St-Félix,641301.2500,6369760.0,0
184,12008_0060_00002,2.0,,Chemin,de la Carrière,12390,Anglars-Saint-Félix,641345.3125,6369750.5,IMB/12008/X/0002,2.0,,Chemin,de la Carrière,12390,Anglars-St-Félix,641353.2500,6369745.5,1
185,12008_0045_00001,1.0,,Chemin,des Bosquets,12390,Anglars-Saint-Félix,641467.0000,6369919.5,IMB/12008/X/000H,1.0,,Chemin,des Bosquets,12390,Anglars-St-Félix,641455.5000,6369919.0,1
186,12300_2470_00059_ter,59.0,ter,Avenue,du Quercy,12200,Villefranche-de-Rouergue,622621.1250,6362129.5,IMB/12300/X/00YI,59.0,T,Avenue,du Quercy,12200,Villefranche-de-Rouergue,622595.5000,6362158.0,1


In [ ]:
B['TypeVoieImmeuble'] = B['TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')

def complement_numero_voie(string) :
  try :
    string = string.lower()
  except :
    pass
  if string == 'bis' :
    string = 'b'
  if string == 'ter' :
    string = 't'
  if string == 'quater' :
    string = 'q'
  return string

A['rep'] = A['rep'].apply(complement_numero_voie)
B['ComplementNumeroVoieImmeuble'] = B['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)


In [ ]:
ban = A.copy()
ipe = B.copy()
labeled = pd.concat([ban,ipe,labeled.iloc[:,-1]],axis=1)
nom_colonnes = {'id': 'ltable_id','numero' : 'ltable_NumeroVoieImmeuble','rep' : 'ltable_ComplementNumeroVoieImmeuble','TypeVoie' : 'ltable_TypeVoieImmeuble','NomVoieReste' : 'ltable_NomVoieImmeuble','code_postal' : 'ltable_CodePostalImmeuble','nom_commune' : 'ltable_CommuneImmeuble','x' : 'ltable_CoordonneeImmeubleX','y' : 'ltable_CoordonneeImmeubleY', 'IdentifiantImmeuble' : 'rtable_id', 'NumeroVoieImmeuble' : 'rtable_NumeroVoieImmeuble', 'ComplementNumeroVoieImmeuble' : 'rtable_ComplementNumeroVoieImmeuble', 'TypeVoieImmeuble' : 'rtable_TypeVoieImmeuble', 'NomVoieImmeuble' : 'rtable_NomVoieImmeuble', 'CodePostalImmeuble' : 'rtable_CodePostalImmeuble', 'CommuneImmeuble': 'rtable_CommuneImmeuble', 'CoordonneeImmeubleX' : 'rtable_CoordonneeImmeubleX', 'CoordonneImmeubleY' : 'rtable_CoordonneeImmeubleY', 'Label' : 'label'}
labeled = labeled.rename(columns = nom_colonnes)

labeled['ltable_TypeVoieImmeuble'] = labeled['ltable_TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')

labeled



,ltable_id,ltable_NumeroVoieImmeuble,ltable_ComplementNumeroVoieImmeuble,ltable_TypeVoieImmeuble,ltable_NomVoieImmeuble,ltable_CodePostalImmeuble,ltable_CommuneImmeuble,ltable_CoordonneeImmeubleX,ltable_CoordonneeImmeubleY,rtable_id,rtable_NumeroVoieImmeuble,rtable_ComplementNumeroVoieImmeuble,rtable_TypeVoieImmeuble,rtable_NomVoieImmeuble,rtable_CodePostalImmeuble,rtable_CommuneImmeuble,rtable_CoordonneeImmeubleX,rtable_CoordonneeImmeubleY,label
0,12002_0puwwe_00001,1.0,,plan,du Foirail,12520,Aguessac,707758.8750,6339741.0,IMB/12001/X/0001,4.0,,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.1250,6361511.0,0
1,12002_ntujmi_00001,1.0,,avenue,de la Brezegue,12520,Aguessac,707676.5000,6339929.5,IMB/12001/X/0006,3.0,,chemin,de Campan,12630,Agen-d'Aveyron,673813.5625,6361964.0,0
2,12002_ntujmi_00002,2.0,,avenue,de la Brezegue,12520,Aguessac,707691.1250,6339931.5,IMB/12001/X/0007,9.0,,chemin,de Campan,12630,Agen-d'Aveyron,673709.7500,6361977.5,0
3,12002_ntujmi_00003,3.0,,avenue,de la Brezegue,12520,Aguessac,707569.0625,6339992.0,IMB/12001/X/000H,14.0,,chemin,de Cantaserp,12630,Agen-d'Aveyron,673441.2500,6362122.0,0
4,12002_ntujmi_00004,4.0,,avenue,de la Brezegue,12520,Aguessac,707643.8125,6339963.5,IMB/12001/X/000I,16.0,,chemin,de Cantaserp,12630,Agen-d'Aveyron,673431.8750,6362076.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,12299_0024_00024,24.0,,chemin,de la Carral,12430,Villefranche-de-Panat,676497.2500,6331368.0,IMB/12008/X/0001,1.0,,chemin,de la Carrière,12390,Anglars-St-Félix,641301.2500,6369760.0,0
184,12008_0060_00002,2.0,,chemin,de la Carrière,12390,Anglars-Saint-Félix,641345.3125,6369750.5,IMB/12008/X/0002,2.0,,chemin,de la Carrière,12390,Anglars-St-Félix,641353.2500,6369745.5,1
185,12008_0045_00001,1.0,,chemin,des Bosquets,12390,Anglars-Saint-Félix,641467.0000,6369919.5,IMB/12008/X/000H,1.0,,chemin,des Bosquets,12390,Anglars-St-Félix,641455.5000,6369919.0,1
186,12300_2470_00059_ter,59.0,t,avenue,du Quercy,12200,Villefranche-de-Rouergue,622621.1250,6362129.5,IMB/12300/X/00YI,59.0,t,avenue,du Quercy,12200,Villefranche-de-Rouergue,622595.5000,6362158.0,1


In [ ]:
nom_colonnes = {'id': 'IdentifiantImmeuble','numero' : 'NumeroVoieImmeuble','rep' : 'ComplementNumeroVoieImmeuble','TypeVoie' : 'TypeVoieImmeuble','NomVoieReste' : 'NomVoieImmeuble','code_postal' : 'CodePostalImmeuble','nom_commune' : 'CommuneImmeuble','x' : 'CoordonneeImmeubleX','y' : 'CoordonneeImmeubleY'}
A = A.rename(columns=nom_colonnes)

In [ ]:
labeled['_id'] = range(len(labeled))

#We need to set key for each table to create metadata so that we can continue the next step
em.set_key(labeled,'_id')
em.set_fk_ltable(labeled, 'ltable_id')
em.set_fk_rtable(labeled, 'rtable_id')
em.set_key(A, 'IdentifiantImmeuble')
em.set_key(B, 'IdentifiantImmeuble')
em.set_ltable(labeled, A)
em.set_rtable(labeled, B)

labeled


,ltable_id,ltable_NumeroVoieImmeuble,ltable_ComplementNumeroVoieImmeuble,ltable_TypeVoieImmeuble,ltable_NomVoieImmeuble,ltable_CodePostalImmeuble,ltable_CommuneImmeuble,ltable_CoordonneeImmeubleX,ltable_CoordonneeImmeubleY,rtable_id,rtable_NumeroVoieImmeuble,rtable_ComplementNumeroVoieImmeuble,rtable_TypeVoieImmeuble,rtable_NomVoieImmeuble,rtable_CodePostalImmeuble,rtable_CommuneImmeuble,rtable_CoordonneeImmeubleX,rtable_CoordonneeImmeubleY,label,_id
0,12002_0puwwe_00001,1.0,,plan,du Foirail,12520,Aguessac,707758.8750,6339741.0,IMB/12001/X/0001,4.0,,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.1250,6361511.0,0,0
1,12002_ntujmi_00001,1.0,,avenue,de la Brezegue,12520,Aguessac,707676.5000,6339929.5,IMB/12001/X/0006,3.0,,chemin,de Campan,12630,Agen-d'Aveyron,673813.5625,6361964.0,0,1
2,12002_ntujmi_00002,2.0,,avenue,de la Brezegue,12520,Aguessac,707691.1250,6339931.5,IMB/12001/X/0007,9.0,,chemin,de Campan,12630,Agen-d'Aveyron,673709.7500,6361977.5,0,2
3,12002_ntujmi_00003,3.0,,avenue,de la Brezegue,12520,Aguessac,707569.0625,6339992.0,IMB/12001/X/000H,14.0,,chemin,de Cantaserp,12630,Agen-d'Aveyron,673441.2500,6362122.0,0,3
4,12002_ntujmi_00004,4.0,,avenue,de la Brezegue,12520,Aguessac,707643.8125,6339963.5,IMB/12001/X/000I,16.0,,chemin,de Cantaserp,12630,Agen-d'Aveyron,673431.8750,6362076.0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,12299_0024_00024,24.0,,chemin,de la Carral,12430,Villefranche-de-Panat,676497.2500,6331368.0,IMB/12008/X/0001,1.0,,chemin,de la Carrière,12390,Anglars-St-Félix,641301.2500,6369760.0,0,173
184,12008_0060_00002,2.0,,chemin,de la Carrière,12390,Anglars-Saint-Félix,641345.3125,6369750.5,IMB/12008/X/0002,2.0,,chemin,de la Carrière,12390,Anglars-St-Félix,641353.2500,6369745.5,1,174
185,12008_0045_00001,1.0,,chemin,des Bosquets,12390,Anglars-Saint-Félix,641467.0000,6369919.5,IMB/12008/X/000H,1.0,,chemin,des Bosquets,12390,Anglars-St-Félix,641455.5000,6369919.0,1,175
186,12300_2470_00059_ter,59.0,t,avenue,du Quercy,12200,Villefranche-de-Rouergue,622621.1250,6362129.5,IMB/12300/X/00YI,59.0,t,avenue,du Quercy,12200,Villefranche-de-Rouergue,622595.5000,6362158.0,1,176


In [ ]:
G = em.sample_table(labeled, labeled.shape[0])

In [ ]:
# match_f = em.get_features_for_matching(A, B, validate_inferred_attr_types = False)
match_f = em.get_features_for_matching(A,B, validate_inferred_attr_types = False)
match_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,IdentifiantImmeuble_IdentifiantImmeuble_lev_dist,IdentifiantImmeuble,IdentifiantImmeuble,None,None,lev_dist,<function IdentifiantImmeuble_IdentifiantImmeuble_lev_dist at 0x7b7c774bb400>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,IdentifiantImmeuble_IdentifiantImmeuble_lev_sim,IdentifiantImmeuble,IdentifiantImmeuble,None,None,lev_sim,<function IdentifiantImmeuble_IdentifiantImmeuble_lev_sim at 0x7b7c774bbeb0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,IdentifiantImmeuble_IdentifiantImmeuble_jar,IdentifiantImmeuble,IdentifiantImmeuble,None,None,jaro,<function IdentifiantImmeuble_IdentifiantImmeuble_jar at 0x7b7c774bb7f0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,IdentifiantImmeuble_IdentifiantImmeuble_jwn,IdentifiantImmeuble,IdentifiantImmeuble,None,None,jaro_winkler,<function IdentifiantImmeuble_IdentifiantImmeuble_jwn at 0x7b7c774ba4d0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,IdentifiantImmeuble_IdentifiantImmeuble_exm,IdentifiantImmeuble,IdentifiantImmeuble,None,None,exact_match,<function IdentifiantImmeuble_IdentifiantImmeuble_exm at 0x7b7c774bb880>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,IdentifiantImmeuble_IdentifiantImmeuble_jac_qgm_3_qgm_3,IdentifiantImmeuble,IdentifiantImmeuble,qgm_3,qgm_3,jaccard,<function IdentifiantImmeuble_IdentifiantImmeuble_jac_qgm_3_qgm_3 at 0x7b7c774ba440>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,NumeroVoieImmeuble,NumeroVoieImmeuble,None,None,exact_match,<function NumeroVoieImmeuble_NumeroVoieImmeuble_exm at 0x7b7c774ba950>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,NumeroVoieImmeuble_NumeroVoieImmeuble_anm,NumeroVoieImmeuble,NumeroVoieImmeuble,None,None,abs_norm,<function NumeroVoieImmeuble_NumeroVoieImmeuble_anm at 0x7b7c774bbb50>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,NumeroVoieImmeuble_NumeroVoieImmeuble_lev_dist,NumeroVoieImmeuble,NumeroVoieImmeuble,None,None,lev_dist,<function NumeroVoieImmeuble_NumeroVoieImmeuble_lev_dist at 0x7b7c774bbbe0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,NumeroVoieImmeuble_NumeroVoieImmeuble_lev_sim,NumeroVoieImmeuble,NumeroVoieImmeuble,None,None,lev_sim,<function NumeroVoieImmeuble_NumeroVoieImmeuble_lev_sim at 0x7b7c774bbe20>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [ ]:
H_train = em.extract_feature_vecs(G, feature_table=match_f , attrs_after=['label'])
H_train

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_id,rtable_id,IdentifiantImmeuble_IdentifiantImmeuble_lev_dist,IdentifiantImmeuble_IdentifiantImmeuble_lev_sim,IdentifiantImmeuble_IdentifiantImmeuble_jar,IdentifiantImmeuble_IdentifiantImmeuble_jwn,IdentifiantImmeuble_IdentifiantImmeuble_exm,IdentifiantImmeuble_IdentifiantImmeuble_jac_qgm_3_qgm_3,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,NumeroVoieImmeuble_NumeroVoieImmeuble_anm,NumeroVoieImmeuble_NumeroVoieImmeuble_lev_dist,NumeroVoieImmeuble_NumeroVoieImmeuble_lev_sim,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_jac_qgm_3_qgm_3,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_cos_dlm_dc0_dlm_dc0,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_jac_dlm_dc0_dlm_dc0,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_lev_dist,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_lev_sim,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_nmw,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_sw,TypeVoieImmeuble_TypeVoieImmeuble_jac_qgm_3_qgm_3,TypeVoieImmeuble_TypeVoieImmeuble_cos_dlm_dc0_dlm_dc0,TypeVoieImmeuble_TypeVoieImmeuble_jac_dlm_dc0_dlm_dc0,TypeVoieImmeuble_TypeVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_dist,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,TypeVoieImmeuble_TypeVoieImmeuble_nmw,TypeVoieImmeuble_TypeVoieImmeuble_sw,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,NomVoieImmeuble_NomVoieImmeuble_cos_dlm_dc0_dlm_dc0,NomVoieImmeuble_NomVoieImmeuble_jac_dlm_dc0_dlm_dc0,NomVoieImmeuble_NomVoieImmeuble_mel,NomVoieImmeuble_NomVoieImmeuble_lev_dist,NomVoieImmeuble_NomVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_nmw,NomVoieImmeuble_NomVoieImmeuble_sw,CodePostalImmeuble_CodePostalImmeuble_exm,CodePostalImmeuble_CodePostalImmeuble_anm,CodePostalImmeuble_CodePostalImmeuble_lev_dist,CodePostalImmeuble_CodePostalImmeuble_lev_sim,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_cos_dlm_dc0_dlm_dc0,CommuneImmeuble_CommuneImmeuble_jac_dlm_dc0_dlm_dc0,CommuneImmeuble_CommuneImmeuble_mel,CommuneImmeuble_CommuneImmeuble_lev_dist,CommuneImmeuble_CommuneImmeuble_lev_sim,CommuneImmeuble_CommuneImmeuble_nmw,CommuneImmeuble_CommuneImmeuble_sw,CoordonneeImmeubleX_CoordonneeImmeubleX_exm,CoordonneeImmeubleX_CoordonneeImmeubleX_anm,CoordonneeImmeubleX_CoordonneeImmeubleX_lev_dist,CoordonneeImmeubleX_CoordonneeImmeubleX_lev_sim,label
0,0,12002_0puwwe_00001,IMB/12001/X/0001,13,0.277778,0.648148,0.648148,0,0.200000,0,0.250000,1.0,0.666667,1.0,1.0,1.0,1.0,0.0,1.0,-0.0,0.0,0.000000,0.0,0.0,0.464286,6.0,0.142857,-2.0,1.0,0.032258,0.000000,0.00,0.568947,15.0,0.210526,-5.0,3.0,0,0.991291,2.0,0.6,0.083333,0.0,0.0,0.623810,11.0,0.214286,-3.0,2.0,0,0.952921,6.0,0.250000,0
1,1,12002_ntujmi_00001,IMB/12001/X/0006,15,0.166667,0.608796,0.608796,0,0.121212,0,0.333333,1.0,0.666667,1.0,1.0,1.0,1.0,0.0,1.0,-0.0,0.0,0.000000,0.0,0.0,0.555556,5.0,0.166667,1.0,1.0,0.125000,0.408248,0.25,0.703704,10.0,0.285714,-1.0,4.0,0,0.991291,2.0,0.6,0.083333,0.0,0.0,0.623810,11.0,0.214286,-3.0,2.0,0,0.952149,7.0,0.222222,0
2,2,12002_ntujmi_00002,IMB/12001/X/0007,15,0.166667,0.608796,0.608796,0,0.090909,0,0.222222,1.0,0.666667,1.0,1.0,1.0,1.0,0.0,1.0,-0.0,0.0,0.000000,0.0,0.0,0.555556,5.0,0.166667,1.0,1.0,0.125000,0.408248,0.25,0.703704,10.0,0.285714,-1.0,4.0,0,0.991291,2.0,0.6,0.083333,0.0,0.0,0.623810,11.0,0.214286,-3.0,2.0,0,0.951983,6.0,0.333333,0
3,3,12002_ntujmi_00003,IMB/12001/X/000H,15,0.166667,0.608796,0.608796,0,0.088235,0,0.214286,2.0,0.500000,1.0,1.0,1.0,1.0,0.0,1.0,-0.0,0.0,0.000000,0.0,0.0,0.555556,5.0,0.166667,1.0,1.0,0.111111,0.408248,0.25,0.711111,9.0,0.357143,3.0,4.0,0,0.991291,2.0,0.6,0.083333,0.0,0.0,0.623810,11.0,0.214286,-3.0,2.0,0,0.951768,8.0,0.111111,0
4,4,12002_ntujmi_00004,IMB/12001/X/000I,15,0.166667,0.608796,0.608796,0,0.088235,0,0.250000,2.0,0.500000,1.0,1.0,1.0,1.0,0.0,1.0,-0.0,0.0,0.000000,0.0,0.0,0.555556,5.0,0.166667,1.0,1.0,0.111111,0.408248,0.25,0.711111,9.0,0.357

In [ ]:
H_train = em.impute_table(H_train, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], strategy='mean')

In [ ]:
dt = em.DTMatcher(max_depth=5)

In [ ]:
train_test = em.split_train_test(H_train, train_proportion=0.7)
devel_set = train_test['train']
eval_set = train_test['test']

In [ ]:
dt.fit(table=devel_set, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='label')


In [ ]:
pred_table = dt.predict(table=eval_set, exclude_attrs=['_id', 'ltable_id', 'rtable_id','label'], target_attr='predicted_labels', append=True, inplace=True)

In [ ]:
pred_table

,_id,ltable_id,rtable_id,IdentifiantImmeuble_IdentifiantImmeuble_lev_dist,IdentifiantImmeuble_IdentifiantImmeuble_lev_sim,IdentifiantImmeuble_IdentifiantImmeuble_jar,IdentifiantImmeuble_IdentifiantImmeuble_jwn,IdentifiantImmeuble_IdentifiantImmeuble_exm,IdentifiantImmeuble_IdentifiantImmeuble_jac_qgm_3_qgm_3,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,NumeroVoieImmeuble_NumeroVoieImmeuble_anm,NumeroVoieImmeuble_NumeroVoieImmeuble_lev_dist,NumeroVoieImmeuble_NumeroVoieImmeuble_lev_sim,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_jac_qgm_3_qgm_3,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_cos_dlm_dc0_dlm_dc0,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_jac_dlm_dc0_dlm_dc0,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_lev_dist,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_lev_sim,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_nmw,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_sw,TypeVoieImmeuble_TypeVoieImmeuble_jac_qgm_3_qgm_3,TypeVoieImmeuble_TypeVoieImmeuble_cos_dlm_dc0_dlm_dc0,TypeVoieImmeuble_TypeVoieImmeuble_jac_dlm_dc0_dlm_dc0,TypeVoieImmeuble_TypeVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_dist,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,TypeVoieImmeuble_TypeVoieImmeuble_nmw,TypeVoieImmeuble_TypeVoieImmeuble_sw,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,NomVoieImmeuble_NomVoieImmeuble_cos_dlm_dc0_dlm_dc0,NomVoieImmeuble_NomVoieImmeuble_jac_dlm_dc0_dlm_dc0,NomVoieImmeuble_NomVoieImmeuble_mel,NomVoieImmeuble_NomVoieImmeuble_lev_dist,NomVoieImmeuble_NomVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_nmw,NomVoieImmeuble_NomVoieImmeuble_sw,CodePostalImmeuble_CodePostalImmeuble_exm,CodePostalImmeuble_CodePostalImmeuble_anm,CodePostalImmeuble_CodePostalImmeuble_lev_dist,CodePostalImmeuble_CodePostalImmeuble_lev_sim,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_cos_dlm_dc0_dlm_dc0,CommuneImmeuble_CommuneImmeuble_jac_dlm_dc0_dlm_dc0,CommuneImmeuble_CommuneImmeuble_mel,CommuneImmeuble_CommuneImmeuble_lev_dist,CommuneImmeuble_CommuneImmeuble_lev_sim,CommuneImmeuble_CommuneImmeuble_nmw,CommuneImmeuble_CommuneImmeuble_sw,CoordonneeImmeubleX_CoordonneeImmeubleX_exm,CoordonneeImmeubleX_CoordonneeImmeubleX_anm,CoordonneeImmeubleX_CoordonneeImmeubleX_lev_dist,CoordonneeImmeubleX_CoordonneeImmeubleX_lev_sim,label,predicted_labels
141,132,12002_0049_00001,IMB/12002/X/00CQ,13.0,0.235294,0.616422,0.616422,0.0,0.093750,1.0,1.000000,0.0,1.000000,1.0,1.0,1.0,1.0,0.0,1.0,-0.0,0.0,0.250000,0.0,0.0,0.777778,1.0,0.666667,2.0,2.0,0.031250,0.000000,0.000000,0.628676,12.0,0.294118,0.0,4.0,1.0,1.000000,0.0,1.0,1.000000,1.0,1.0,1.000000,0.0,1.000000,8.0,8.0,0.0,0.999604,4.0,0.555556,0.0,0.0
164,154,12255_0172_00441,IMB/12255/X/00DX,13.0,0.187500,0.625000,0.625000,0.0,0.090909,1.0,1.000000,0.0,1.000000,1.0,1.0,1.0,1.0,0.0,1.0,-0.0,0.0,0.454545,0.0,0.0,0.888889,1.0,0.833333,5.0,5.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,8.0,8.0,1.0,1.000000,0.0,1.0,1.000000,1.0,1.0,1.000000,0.0,1.000000,8.0,8.0,0.0,1.000000,3.0,0.666667,1.0,1.0
114,109,12002_2rjm3p_00003,IMB/12002/X/00E9,15.0,0.166667,0.608796,0.608796,0.0,0.088235,0.0,0.272727,2.0,0.500000,1.0,1.0,1.0,1.0,0.0,1.0,-0.0,0.0,1.000000,1.0,1.0,1.000000,0.0,1.000000,-0.0,0.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,8.0,8.0,1.0,1.000000,0.0,1.0,1.000000,1.0,1.0,1.000000,0.0,1.000000,8.0,8.0,0.0,0.999591,4.0,0.555556,0.0,0.0
36,36,12013_0892_00034,IMB/12013/X/004I,13.0,0.187500,0.666667,0.666667,0.0,0.090909,0.0,0.411765,1.0,0.750000,1.0,1.0,1.0,1.0,0.0,1.0,-0.0,0.0,0.076923,0.0,0.0,0.733333,2.0,0.600000,3.0,3.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,8.0,8.0,1.0,1.000000,0.0,1.0,1.000000,1.0,1.0,1.000000,0.0,1.000000,5.0,5.0,0.0,0.999817,5.0,0.444444,0.0,0.0
81,81,12012_j7y8bu_00006,IMB/12012/X/007F,15.0,0.166667,0.606481,0.606481,0.0,0.088235,1.0,1.000000,0.0,1.000000,1.0,1.0,1.0,1.0,0.0,1

In [ ]:
pred_table[pred_table['label']==1.0]

,_id,ltable_id,rtable_id,IdentifiantImmeuble_IdentifiantImmeuble_lev_dist,IdentifiantImmeuble_IdentifiantImmeuble_lev_sim,IdentifiantImmeuble_IdentifiantImmeuble_jar,IdentifiantImmeuble_IdentifiantImmeuble_jwn,IdentifiantImmeuble_IdentifiantImmeuble_exm,IdentifiantImmeuble_IdentifiantImmeuble_jac_qgm_3_qgm_3,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,NumeroVoieImmeuble_NumeroVoieImmeuble_anm,NumeroVoieImmeuble_NumeroVoieImmeuble_lev_dist,NumeroVoieImmeuble_NumeroVoieImmeuble_lev_sim,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_jac_qgm_3_qgm_3,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_cos_dlm_dc0_dlm_dc0,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_jac_dlm_dc0_dlm_dc0,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_lev_dist,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_lev_sim,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_nmw,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_sw,TypeVoieImmeuble_TypeVoieImmeuble_jac_qgm_3_qgm_3,TypeVoieImmeuble_TypeVoieImmeuble_cos_dlm_dc0_dlm_dc0,TypeVoieImmeuble_TypeVoieImmeuble_jac_dlm_dc0_dlm_dc0,TypeVoieImmeuble_TypeVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_dist,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,TypeVoieImmeuble_TypeVoieImmeuble_nmw,TypeVoieImmeuble_TypeVoieImmeuble_sw,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,NomVoieImmeuble_NomVoieImmeuble_cos_dlm_dc0_dlm_dc0,NomVoieImmeuble_NomVoieImmeuble_jac_dlm_dc0_dlm_dc0,NomVoieImmeuble_NomVoieImmeuble_mel,NomVoieImmeuble_NomVoieImmeuble_lev_dist,NomVoieImmeuble_NomVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_nmw,NomVoieImmeuble_NomVoieImmeuble_sw,CodePostalImmeuble_CodePostalImmeuble_exm,CodePostalImmeuble_CodePostalImmeuble_anm,CodePostalImmeuble_CodePostalImmeuble_lev_dist,CodePostalImmeuble_CodePostalImmeuble_lev_sim,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_cos_dlm_dc0_dlm_dc0,CommuneImmeuble_CommuneImmeuble_jac_dlm_dc0_dlm_dc0,CommuneImmeuble_CommuneImmeuble_mel,CommuneImmeuble_CommuneImmeuble_lev_dist,CommuneImmeuble_CommuneImmeuble_lev_sim,CommuneImmeuble_CommuneImmeuble_nmw,CommuneImmeuble_CommuneImmeuble_sw,CoordonneeImmeubleX_CoordonneeImmeubleX_exm,CoordonneeImmeubleX_CoordonneeImmeubleX_anm,CoordonneeImmeubleX_CoordonneeImmeubleX_lev_dist,CoordonneeImmeubleX_CoordonneeImmeubleX_lev_sim,label,predicted_labels
164,154,12255_0172_00441,IMB/12255/X/00DX,13.0,0.187500,0.625000,0.625000,0.0,0.090909,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,-0.0,0.0,0.454545,0.0,0.0,0.888889,1.0,0.833333,5.0,5.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,8.0,8.0,1.0,1.0,0.0,1.0,1.000000,1.0,1.0,1.000000,0.0,1.000000,8.0,8.0,0.0,1.0,3.0,0.666667,1.0,1.0
81,81,12012_j7y8bu_00006,IMB/12012/X/007F,15.0,0.166667,0.606481,0.606481,0.0,0.088235,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,-0.0,0.0,0.625000,0.0,0.0,0.939394,1.0,0.909091,10.0,10.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,11.0,11.0,1.0,1.0,0.0,1.0,1.000000,1.0,1.0,1.000000,0.0,1.000000,9.0,9.0,1.0,1.0,0.0,1.000000,1.0,1.0
184,174,12008_0060_00002,IMB/12008/X/0002,10.0,0.375000,0.708333,0.708333,0.0,0.259259,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,-0.0,0.0,0.454545,0.0,0.0,0.888889,1.0,0.833333,5.0,5.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,14.0,14.0,1.0,1.0,0.0,1.0,0.695652,0.0,0.0,0.930921,3.0,0.842105,13.0,13.0,0.0,1.0,4.0,0.555556,1.0,1.0
129,124,12224_lxqsfe_00001,IMB/12224/X/005M,15.0,0.166667,0.608796,0.608796,0.0,0.088235,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,-0.0,0.0,0.500000,0.0,0.0,0.904762,1.0,0.857143,6.0,6.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,12.0,12.0,1.0,1.0,0.0,1.0,0.372093,0.0,0.0,0.859469,7.0,0.766667,20.0,22.0,1.0,1.0,0.0,1.000000,1.0,1.0
128,123,12224_lxqsfe_00005,IMB/12224/X/005L,15.0,0.166667,0.648148,0.648148,0.0,0.121212,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,-0.0,0.0,0.500000,0.0,0.0,0.904762,1.0,0.857143,6.0,6.0,1.000000,1.000000

In [ ]:
eval_summary = em.eval_matches(pred_table, 'label', 'predicted_labels')
eval_summary

OrderedDict([('prec_numerator', 24.0),
             ('prec_denominator', 24.0),
             ('precision', 1.0),
             ('recall_numerator', 24.0),
             ('recall_denominator', 24.0),
             ('recall', 1.0),
             ('f1', 1.0),
             ('pred_pos_num', 24.0),
             ('false_pos_num', 0.0),
             ('false_pos_ls', []),
             ('pred_neg_num', 30.0),
             ('false_neg_num', 0.0),
             ('false_neg_ls', [])])

In [ ]:
'''
dt = em.DTMatcher()
rf = em.RFMatcher()
result = em.select_matcher(matchers=[dt, rf], table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='label', k=5)
'''

"\ndt = em.DTMatcher()\nrf = em.RFMatcher()\nresult = em.select_matcher(matchers=[dt, rf], table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='label', k=5)\n"

# Prediction

Dans cette partie, nous utilisons le modèle entrainé sur le dataset annoté d'entrainement afin de prédire si les adresses sont similaires ou non.

In [ ]:
#si vous avez enregistré les dataframes de la section précédente sous forme de CSV pour les réutiliser, vous pouvez les charger ici en précisant le chemin des CSV.
ban_reduced = pd.read_csv('/content/ban_reduced.csv')
ipe_reduced = pd.read_csv('/content/ipe_reduced.csv')

In [ ]:
A = ipe_reduced.copy()
B = ban_reduced.copy()

#A.rename(columns={'IdentifiantImmeuble': 'id'}, inplace=True)
#B.rename(columns={'IdentifiantImmeuble': 'id'}, inplace=True)

em.set_key(A, 'IdentifiantImmeuble')
em.set_key(B, 'IdentifiantImmeuble')


shape_A = A.shape
shape_B = B.shape
print(shape_A)
print(shape_B)

(283831, 10)
(152683, 10)


In [ ]:
def complement_numero_voie(string) :
  try :
    string = string.lower()
  except :
    pass
  if string == 'bis' :
    string = 'b'
  if string == 'ter' :
    string = 't'
  if string == 'quater' :
    string = 'q'
  return string

A['ComplementNumeroVoieImmeuble'] = A['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)
B['ComplementNumeroVoieImmeuble'] = B['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)


### Blocking

Comme pour l'algorithme 1 basé sur nos propres règles, nous définissons des colonnes pour lesquelles la correspondance doit être stricte entre les 2 adresses :
- NumeroVoieImmeuble
- ComplementNumeroVoieImmeuble
- TypeVoieImmeuble
- CodePostalImmeuble


In [ ]:
# define the blocker attributes to check and put in 1 column (because AttrEquivalenceBlocker can only handle 1 column comparison)
# for the unknown NumeroVoieImmeuble, we impute with the value 0
'''
A['NumeroVoieImmeuble'] = A['NumeroVoieImmeuble'].fillna(value = 0).astype(int)
print('We imputed', len(A[A['NumeroVoieImmeuble'] == 0]), 'unknown NumeroVoieImmeuble in the IPE dataset')

B['NumeroVoieImmeuble'] = B['NumeroVoieImmeuble'].fillna(value = 0).astype(int)
print('We imputed', len(B[B['NumeroVoieImmeuble'] == 0]), 'unknown NumeroVoieImmeuble in the BAN dataset')

A['CodePostalImmeuble'] = A['CodePostalImmeuble'].fillna(value = 00000).astype(int)
print('We imputed', len(A[A['CodePostalImmeuble'] == 0]), 'unknown CodePostalImmeuble in the IPE dataset')

B['CodePostalImmeuble'] = B['CodePostalImmeuble'].fillna(value = 00000).astype(int)
print('We imputed', len(B[B['CodePostalImmeuble'] == 0]), 'unknown CodePostalImmeuble in the BAN dataset')
'''



A['blocking_data'] = A['NumeroVoieImmeuble'].fillna('').astype(str)+ ' '  +  A['ComplementNumeroVoieImmeuble'].fillna('').astype(str) + ' ' + A['TypeVoieImmeuble'].fillna('').astype(str) + ' ' + A['CodePostalImmeuble'].fillna('').astype(str)
B['blocking_data'] = B['NumeroVoieImmeuble'].fillna('').astype(str)+ ' '  +  B['ComplementNumeroVoieImmeuble'].fillna('').astype(str) + ' ' + B['TypeVoieImmeuble'].fillna('').astype(str) + ' ' + B['CodePostalImmeuble'].fillna('').astype(str)



In [ ]:
attributes = ['IdentifiantImmeuble','NumeroVoieImmeuble','ComplementNumeroVoieImmeuble','TypeVoieImmeuble','NomVoieImmeuble','CodePostalImmeuble','CommuneImmeuble','CoordonneeImmeubleX','CoordonneeImmeubleY']

ab = em.AttrEquivalenceBlocker()

# Use block_tables to apply blocking over two input tables.
C = ab.block_tables(A, B,
                    l_block_attr='blocking_data', r_block_attr='blocking_data',
                    l_output_attrs=attributes,
                    r_output_attrs=attributes,
                    l_output_prefix='ltable_', r_output_prefix='rtable_')

C

,_id,ltable_IdentifiantImmeuble,rtable_IdentifiantImmeuble,ltable_NumeroVoieImmeuble,ltable_ComplementNumeroVoieImmeuble,ltable_TypeVoieImmeuble,ltable_NomVoieImmeuble,ltable_CodePostalImmeuble,ltable_CommuneImmeuble,ltable_CoordonneeImmeubleX,ltable_CoordonneeImmeubleY,rtable_NumeroVoieImmeuble,rtable_ComplementNumeroVoieImmeuble,rtable_TypeVoieImmeuble,rtable_NomVoieImmeuble,rtable_CodePostalImmeuble,rtable_CommuneImmeuble,rtable_CoordonneeImmeubleX,rtable_CoordonneeImmeubleY
0,0,IMB/12001/X/0001,12001_0052_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,Adrien Vezinhet,12630,Agen-d'Aveyron,673832.68,6362135.56
1,1,IMB/12001/X/0001,12001_0040_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,Daude de Prades,12630,Agen-d'Aveyron,673699.44,6362045.91
2,2,IMB/12001/X/0001,12001_0083_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674471.55,6361516.82
3,3,IMB/12001/X/0001,12001_0067_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,de la Cascade,12630,Agen-d'Aveyron,677171.63,6360381.52
4,4,IMB/12001/X/0001,12001_0069_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06,4,NaN,impasse,de la Croux,12630,Agen-d'Aveyron,674318.84,6362009.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,408,NaN,chemin,de Bellevue,12410,Curan,689708.742599,6.344055e+06,408,NaN,chemin,de Bellevue,12410,Curan,689727.62,6344090.54
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688529.268230,6.344332e+06,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688560.571568,6.344372e+06,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,349,NaN,chemin,de Martials,12410,Curan,689480.028156,6.347026e+06,349,NaN,chemin,de Martials,12410,Curan,689478.05,6347044.43


### ML prediction

In [ ]:
H = em.extract_feature_vecs(C, feature_table=match_f)
H = em.impute_table(H, exclude_attrs=['_id', 'ltable_IdentifiantImmeuble', 'rtable_IdentifiantImmeuble'], strategy='mean')
H

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:30:21


,_id,ltable_IdentifiantImmeuble,rtable_IdentifiantImmeuble,IdentifiantImmeuble_IdentifiantImmeuble_lev_dist,IdentifiantImmeuble_IdentifiantImmeuble_lev_sim,IdentifiantImmeuble_IdentifiantImmeuble_jar,IdentifiantImmeuble_IdentifiantImmeuble_jwn,IdentifiantImmeuble_IdentifiantImmeuble_exm,IdentifiantImmeuble_IdentifiantImmeuble_jac_qgm_3_qgm_3,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,NumeroVoieImmeuble_NumeroVoieImmeuble_anm,NumeroVoieImmeuble_NumeroVoieImmeuble_lev_dist,NumeroVoieImmeuble_NumeroVoieImmeuble_lev_sim,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_jac_qgm_3_qgm_3,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_cos_dlm_dc0_dlm_dc0,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_jac_dlm_dc0_dlm_dc0,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_lev_dist,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_lev_sim,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_nmw,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_sw,TypeVoieImmeuble_TypeVoieImmeuble_jac_qgm_3_qgm_3,TypeVoieImmeuble_TypeVoieImmeuble_cos_dlm_dc0_dlm_dc0,TypeVoieImmeuble_TypeVoieImmeuble_jac_dlm_dc0_dlm_dc0,TypeVoieImmeuble_TypeVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_dist,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,TypeVoieImmeuble_TypeVoieImmeuble_nmw,TypeVoieImmeuble_TypeVoieImmeuble_sw,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,NomVoieImmeuble_NomVoieImmeuble_cos_dlm_dc0_dlm_dc0,NomVoieImmeuble_NomVoieImmeuble_jac_dlm_dc0_dlm_dc0,NomVoieImmeuble_NomVoieImmeuble_mel,NomVoieImmeuble_NomVoieImmeuble_lev_dist,NomVoieImmeuble_NomVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_nmw,NomVoieImmeuble_NomVoieImmeuble_sw,CodePostalImmeuble_CodePostalImmeuble_exm,CodePostalImmeuble_CodePostalImmeuble_anm,CodePostalImmeuble_CodePostalImmeuble_lev_dist,CodePostalImmeuble_CodePostalImmeuble_lev_sim,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_cos_dlm_dc0_dlm_dc0,CommuneImmeuble_CommuneImmeuble_jac_dlm_dc0_dlm_dc0,CommuneImmeuble_CommuneImmeuble_mel,CommuneImmeuble_CommuneImmeuble_lev_dist,CommuneImmeuble_CommuneImmeuble_lev_sim,CommuneImmeuble_CommuneImmeuble_nmw,CommuneImmeuble_CommuneImmeuble_sw,CoordonneeImmeubleX_CoordonneeImmeubleX_exm,CoordonneeImmeubleX_CoordonneeImmeubleX_anm,CoordonneeImmeubleX_CoordonneeImmeubleX_lev_dist,CoordonneeImmeubleX_CoordonneeImmeubleX_lev_sim
0,0,IMB/12001/X/0001,12001_0052_00004,10.0,0.3750,0.666667,0.666667,0.0,0.137931,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,7.0,7.0,0.000000,0.000000,0.000000,0.516374,17.0,0.105263,-3.0,3.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,14.0,14.0,0.0,0.999102,10.0,0.333333
1,1,IMB/12001/X/0001,12001_0040_00004,10.0,0.3750,0.666667,0.666667,0.0,0.142857,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,7.0,7.0,0.028571,0.288675,0.166667,0.597661,14.0,0.263158,-2.0,5.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,14.0,14.0,0.0,0.998905,9.0,0.400000
2,2,IMB/12001/X/0001,12001_0083_00004,10.0,0.3750,0.666667,0.666667,0.0,0.137931,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,7.0,7.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,19.0,19.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,14.0,14.0,0.0,1.000000,9.0,0.400000
3,3,IMB/12001/X/0001,12001_0067_00004,10.0,0.3750,0.666667,0.666667,0.0,0.137931,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,7.0,7.0,0.250000,0.577350,0.400000,0.839109,10.0,0.473684,3.0,8.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,14.0,14.0,0.0,0.995963,11.0,0.266667
4,4,IMB/12001/X/0001,12001_0069_00004,10.0,0.3750,0.666667,0.666667,0.0,0.137931,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,7.0,7.0,0.222222,0.577350,0.400000,0.829665,11.0,0.421053,0.0,6.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,14.0,14.0,0.0,0.999823,8.0,0.466667
...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
print(devel_set.columns)
print(H.columns)

Index(['_id', 'ltable_id', 'rtable_id',
       'IdentifiantImmeuble_IdentifiantImmeuble_lev_dist',
       'IdentifiantImmeuble_IdentifiantImmeuble_lev_sim',
       'IdentifiantImmeuble_IdentifiantImmeuble_jar',
       'IdentifiantImmeuble_IdentifiantImmeuble_jwn',
       'IdentifiantImmeuble_IdentifiantImmeuble_exm',
       'IdentifiantImmeuble_IdentifiantImmeuble_jac_qgm_3_qgm_3',
       'NumeroVoieImmeuble_NumeroVoieImmeuble_exm',
       'NumeroVoieImmeuble_NumeroVoieImmeuble_anm',
       'NumeroVoieImmeuble_NumeroVoieImmeuble_lev_dist',
       'NumeroVoieImmeuble_NumeroVoieImmeuble_lev_sim',
       'ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_jac_qgm_3_qgm_3',
       'ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_cos_dlm_dc0_dlm_dc0',
       'ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_jac_dlm_dc0_dlm_dc0',
       'ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel',
       'ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_

C'est dans cette cellule que nous utilisons notre modèle de DecisionTree  afin d'évaluer si 2 adresses sont similaires ou non.
Si les adresses sont similaires, predicted_labels = 1, et 0 sinon.
Le désavantage de cette méthode est que la prédiciton est binaire, et donc contrairement au premier algorithme proposé, on n'obtient pas réellement un % de confiance de la prédiction, même si nous pouvons le recalculer grâce aux mêmes méthodes que dans le 1er algorithme (Levensthein distance).

En réalité, cet algorithme de machine learning se base aussi sur des calculs de distance entre les différentes composantes de l'adresse, mais il utilise un processus d'arbre de décision afin d'optimiser la sélection.

In [ ]:
#dt.fit(table=devel_set, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='label')
#pred_table = dt.predict(table=H, exclude_attrs=['_id', 'ltable_IdentifiantImmeuble', 'rtable_IdentifiantImmeuble','predicted_labels'], target_attr='predicted_labels', append=True, inplace=True)
pred_table = dt.predict(table=H, exclude_attrs=['_id', 'ltable_IdentifiantImmeuble', 'rtable_IdentifiantImmeuble'], target_attr='predicted_labels', append=True, inplace=True)

pred_table


,_id,ltable_IdentifiantImmeuble,rtable_IdentifiantImmeuble,IdentifiantImmeuble_IdentifiantImmeuble_lev_dist,IdentifiantImmeuble_IdentifiantImmeuble_lev_sim,IdentifiantImmeuble_IdentifiantImmeuble_jar,IdentifiantImmeuble_IdentifiantImmeuble_jwn,IdentifiantImmeuble_IdentifiantImmeuble_exm,IdentifiantImmeuble_IdentifiantImmeuble_jac_qgm_3_qgm_3,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,NumeroVoieImmeuble_NumeroVoieImmeuble_anm,NumeroVoieImmeuble_NumeroVoieImmeuble_lev_dist,NumeroVoieImmeuble_NumeroVoieImmeuble_lev_sim,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_jac_qgm_3_qgm_3,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_cos_dlm_dc0_dlm_dc0,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_jac_dlm_dc0_dlm_dc0,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_lev_dist,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_lev_sim,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_nmw,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_sw,TypeVoieImmeuble_TypeVoieImmeuble_jac_qgm_3_qgm_3,TypeVoieImmeuble_TypeVoieImmeuble_cos_dlm_dc0_dlm_dc0,TypeVoieImmeuble_TypeVoieImmeuble_jac_dlm_dc0_dlm_dc0,TypeVoieImmeuble_TypeVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_dist,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,TypeVoieImmeuble_TypeVoieImmeuble_nmw,TypeVoieImmeuble_TypeVoieImmeuble_sw,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,NomVoieImmeuble_NomVoieImmeuble_cos_dlm_dc0_dlm_dc0,NomVoieImmeuble_NomVoieImmeuble_jac_dlm_dc0_dlm_dc0,NomVoieImmeuble_NomVoieImmeuble_mel,NomVoieImmeuble_NomVoieImmeuble_lev_dist,NomVoieImmeuble_NomVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_nmw,NomVoieImmeuble_NomVoieImmeuble_sw,CodePostalImmeuble_CodePostalImmeuble_exm,CodePostalImmeuble_CodePostalImmeuble_anm,CodePostalImmeuble_CodePostalImmeuble_lev_dist,CodePostalImmeuble_CodePostalImmeuble_lev_sim,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_cos_dlm_dc0_dlm_dc0,CommuneImmeuble_CommuneImmeuble_jac_dlm_dc0_dlm_dc0,CommuneImmeuble_CommuneImmeuble_mel,CommuneImmeuble_CommuneImmeuble_lev_dist,CommuneImmeuble_CommuneImmeuble_lev_sim,CommuneImmeuble_CommuneImmeuble_nmw,CommuneImmeuble_CommuneImmeuble_sw,CoordonneeImmeubleX_CoordonneeImmeubleX_exm,CoordonneeImmeubleX_CoordonneeImmeubleX_anm,CoordonneeImmeubleX_CoordonneeImmeubleX_lev_dist,CoordonneeImmeubleX_CoordonneeImmeubleX_lev_sim,predicted_labels
0,0,IMB/12001/X/0001,12001_0052_00004,10.0,0.3750,0.666667,0.666667,0.0,0.137931,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,7.0,7.0,0.000000,0.000000,0.000000,0.516374,17.0,0.105263,-3.0,3.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,14.0,14.0,0.0,0.999102,10.0,0.333333,0.0
1,1,IMB/12001/X/0001,12001_0040_00004,10.0,0.3750,0.666667,0.666667,0.0,0.142857,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,7.0,7.0,0.028571,0.288675,0.166667,0.597661,14.0,0.263158,-2.0,5.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,14.0,14.0,0.0,0.998905,9.0,0.400000,0.0
2,2,IMB/12001/X/0001,12001_0083_00004,10.0,0.3750,0.666667,0.666667,0.0,0.137931,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,7.0,7.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,19.0,19.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,14.0,14.0,0.0,1.000000,9.0,0.400000,1.0
3,3,IMB/12001/X/0001,12001_0067_00004,10.0,0.3750,0.666667,0.666667,0.0,0.137931,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,7.0,7.0,0.250000,0.577350,0.400000,0.839109,10.0,0.473684,3.0,8.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,14.0,14.0,0.0,0.995963,11.0,0.266667,0.0
4,4,IMB/12001/X/0001,12001_0069_00004,10.0,0.3750,0.666667,0.666667,0.0,0.137931,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,7.0,7.0,0.222222,0.577350,0.400000,0.829665,11.0,0.421053,0.0,6.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,14.0,14.0,0.0,0.999823,8.0,0.466667,0.0
...,...,...,...,

In [ ]:
print('We found', len(pred_table[pred_table['predicted_labels'] == 1]), 'matching adresses')

We found 65904 matching adresses
